In [1]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights, densenet121, DenseNet121_Weights
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from stratified_dataset_splitter import StratifiedDatasetSplitter
from early_stopping import EarlyStopping
from evaluate_model import evaluate_model, print_evaluation_results
from seed import set_global_seed

In [2]:
# Configurations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 3
num_epochs = 100
batch_size = 32
learning_rate = 5e-5
dropout_rate = 0.5
weight_decay = 1e-4
set_global_seed()

In [3]:
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

densenet = densenet121(weights=DenseNet121_Weights.DEFAULT)
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)

models = {
    'resnet': resnet,
    'densenet': densenet,
    'deit': timm.create_model("deit_base_patch16_224", pretrained=True, num_classes=num_classes),
}

for model in models.values():
    model = model.to("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Split dataset
splitter = StratifiedDatasetSplitter(data_dir='MSID3')
train_loader, val_loader, test_loader = splitter.create_dataloaders()

In [5]:
def evaluate(model, dataloader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(dataloader)
    accuracy = 100 * correct / total
    return val_loss, accuracy


In [6]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [7]:
evaluation_results = {}

for model_name, model in models.items():
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5)
    early_stopping = EarlyStopping(patience=15, min_delta=0.01)

    print(f"Training {model_name}...")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
    
        # Progress bar for training
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
    
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        val_loss, val_accuracy = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    
        print(f"Last learning rate: {scheduler.get_last_lr()}")
        # Adjust learning rate
        scheduler.step(val_loss)
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break
    
    print("Training complete!")

    # model.eval()
    # correct = 0
    # total = 0
    
    # with torch.no_grad():
    #     pbar = tqdm(test_loader)
    #     for images, labels in pbar:
    #         images, labels = images.to(device), labels.to(device)
    #         outputs = model(images)
    #         _, predicted = torch.max(outputs, 1)
    #         correct += (predicted == labels).sum().item()
    #         total += labels.size(0)
    
    # accuracy = 100 * correct / total
    # print(f"{variation} Testing Accuracy: {accuracy:.2f}%")

    # Evaluate the model
    results = evaluate_model(
        model=model, 
        dataloader=test_loader, 
        device=device,
        multiclass=True  # Set based on your problem
    )
    
    # Print results
    print(f"Test results for {model_name}")
    print_evaluation_results(results)

    evaluation_results[model_name] = results

Training resnet...


Epoch 1/100: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.97it/s]


Epoch 1/100, Train Loss: 0.8520, Val Loss: 0.6398, Val Accuracy: 75.00%
Last learning rate: [5e-05]


Epoch 2/100: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  6.67it/s]


Epoch 2/100, Train Loss: 0.5150, Val Loss: 0.4666, Val Accuracy: 81.45%
Last learning rate: [5e-05]


...

Epoch 25/100, Train Loss: 0.0299, Val Loss: 0.2324, Val Accuracy: 94.35%
Last learning rate: [5.000000000000001e-07]


Epoch 26/100: 100%|█████████████████████████████████████████████████████████████████████| 16/16 [01:17<00:00,  4.86s/it]


Epoch 26/100, Train Loss: 0.0438, Val Loss: 0.2306, Val Accuracy: 94.35%
Last learning rate: [5.000000000000001e-07]


Epoch 27/100: 100%|█████████████████████████████████████████████████████████████████████| 16/16 [01:18<00:00,  4.92s/it]


Epoch 27/100, Train Loss: 0.0251, Val Loss: 0.2295, Val Accuracy: 94.35%
Last learning rate: [5.000000000000001e-07]
Early stopping triggered.
Training complete!
Test results for deit
Model Evaluation Metrics:
Accuracy: 0.9221
Precision: 0.9212
Recall: 0.9221
F1 Score: 0.9209

Confusion Matrix:
[58, 0, 0]
[1, 51, 4]
[3, 4, 33]

Detailed Classification Report:
0:
  precision: 0.9354838709677419
  recall: 1.0
  f1-score: 0.9666666666666667
  support: 58.0
1:
  precision: 0.9272727272727272
  recall: 0.9107142857142857
  f1-score: 0.918918918918919
  support: 56.0
2:
  precision: 0.8918918918918919
  recall: 0.825
  f1-score: 0.8571428571428571
  support: 40.0


In [8]:
print(evaluation_results)

{'resnet': {'accuracy': 0.8766233766233766, 'confusion_matrix': [[58, 0, 0], [4, 44, 8], [4, 3, 33]], 'classification_report': {'0': {'precision': 0.8787878787878788, 'recall': 1.0, 'f1-score': 0.9354838709677419, 'support': 58.0}, '1': {'precision': 0.9361702127659575, 'recall': 0.7857142857142857, 'f1-score': 0.8543689320388349, 'support': 56.0}, '2': {'precision': 0.8048780487804879, 'recall': 0.825, 'f1-score': 0.8148148148148148, 'support': 40.0}, 'accuracy': 0.8766233766233766, 'macro avg': {'precision': 0.8732787134447747, 'recall': 0.8702380952380953, 'f1-score': 0.8682225392737971, 'support': 154.0}, 'weighted avg': {'precision': 0.8804568236091564, 'recall': 0.8766233766233766, 'f1-score': 0.8746449175512752, 'support': 154.0}}, 'precision': 0.8804568236091564, 'recall': 0.8766233766233766, 'f1_score': 0.8746449175512752, 'roc_auc': None}, 'densenet': {'accuracy': 0.9155844155844156, 'confusion_matrix': [[58, 0, 0], [3, 48, 5], [2, 3, 35]], 'classification_report': {'0': {'pr

In [9]:
evaluation_results

{'resnet': {'accuracy': 0.8766233766233766,
  'confusion_matrix': [[58, 0, 0], [4, 44, 8], [4, 3, 33]],
  'classification_report': {'0': {'precision': 0.8787878787878788,
    'recall': 1.0,
    'f1-score': 0.9354838709677419,
    'support': 58.0},
   '1': {'precision': 0.9361702127659575,
    'recall': 0.7857142857142857,
    'f1-score': 0.8543689320388349,
    'support': 56.0},
   '2': {'precision': 0.8048780487804879,
    'recall': 0.825,
    'f1-score': 0.8148148148148148,
    'support': 40.0},
   'accuracy': 0.8766233766233766,
   'macro avg': {'precision': 0.8732787134447747,
    'recall': 0.8702380952380953,
    'f1-score': 0.8682225392737971,
    'support': 154.0},
   'weighted avg': {'precision': 0.8804568236091564,
    'recall': 0.8766233766233766,
    'f1-score': 0.8746449175512752,
    'support': 154.0}},
  'precision': 0.8804568236091564,
  'recall': 0.8766233766233766,
  'f1_score': 0.8746449175512752,
  'roc_auc': None},
 'densenet': {'accuracy': 0.9155844155844156,
  'co